1. 超参数设置：

In [ ]:
lr = 0.01
n_epoch = 200
hidden_dim = 16
l2_coef = 5e-4
dataset = 'cora'
dataset_path = './examples/gcn/'
best_model_path = './'
self_loops = 1
gpu = -1
if gpu >= 0:
    tlx.set_device("GPU", gpu)
else:
    tlx.set_device("CPU")

2. 数据集加载和预处理

In [ ]:
import gammagl.transforms as T
from gammagl.datasets import IMDB
from gammagl.utils import mask_to_index, set_device

metapaths = [[('movie', 'actor'), ('actor', 'movie')],
    [('movie', 'director'), ('director', 'movie')]]
transform = T.AddMetaPaths(metapaths=metapaths, drop_orig_edges=True,
drop_unconnected_nodes=True)
dataset = IMDB(args.dataset_path, transform=transform)
graph = dataset[0]
y = graph['movie'].y

train_idx = mask_to_index(graph['movie'].train_mask, )
test_idx = mask_to_index(graph['movie'].test_mask)
val_idx = mask_to_index(graph['movie'].val_mask)

edge_index_dict = {}
if tlx.BACKEND == 'tensorflow':
    edge_index_dict = graph.edge_index_dict
else:
    edge_index_dict[('movie', 'metapath_0', 'movie')] = tlx.convert_to_tensor(
        graph.edge_index_dict[('movie', 'metapath_0', 'movie')], dtype=tlx.int64)
    edge_index_dict[('movie', 'metapath_1', 'movie')] = tlx.convert_to_tensor(
        graph.edge_index_dict[('movie', 'metapath_1', 'movie')], dtype=tlx.int64)
data = {
"x_dict": graph.x_dict,
"y": y,
"edge_index_dict": edge_index_dict,
"train_idx": train_idx,
"test_idx": test_idx,
"val_idx": val_idx,
"num_nodes_dict": {'movie': graph['movie'].num_nodes},
}


3. 卷积层构建

In [ ]:
import tensorlayerx as tlx
from tensorlayerx.nn import Module, Sequential, ModuleDict, Linear, Tanh
from gammagl.layers.conv import MessagePassing
from gammagl.layers.conv import GATConv

# SemAttAggr类实现了一个聚合模块，用于通过注意力机制聚合来自不同邻居节点的特征。
class SemAttAggr(Module):
    def __init__(self, in_size, hidden_size):
        super().__init__()

        self.project = Sequential(
            Linear(in_features=in_size, out_features=hidden_size),
            Tanh(),
            Linear(in_features=hidden_size, out_features=1, b_init=None)
        ) # 定义了一个线性投影，通过Tanh激活函数进行变换。

    def forward(self, z):
        w = tlx.reduce_mean(self.project(z), axis=1) 
        beta = tlx.softmax(w, axis=0)
        beta = tlx.expand_dims(beta, axis=-1)
        return tlx.reduce_sum(beta * z, axis=0) # 加权聚合特征

class HANConv(MessagePassing):
    def __init__(self,
                 in_channels,
                 out_channels,
                 metadata,
                 heads=1,
                 negative_slope=0.2):
        super().__init__()
        if not isinstance(in_channels, dict):
            in_channels = {node_type: in_channels for node_type in metadata[0]}
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.metadata = metadata
        self.heads = heads
        self.negetive_slop = negative_slope
        self.gat_dict = ModuleDict({}) # 用来初始化每种边类型的GATConv层。
        for edge_type in metadata[1]:
            src_type, _, dst_type = edge_type
            edge_type = '__'.join(edge_type)
            self.gat_dict[edge_type] = GATConv(in_channels=in_channels[src_type],
                                               out_channels=out_channels,
                                               heads=heads,
                                               concat=True)
        self.sem_att_aggr = SemAttAggr(in_size=out_channels*heads,
                                       hidden_size=out_channels) # 用于计算每个节点类型的特征加权和来聚合信息

    def forward(self, x_dict, edge_index_dict, num_nodes_dict):
        out_dict = {}  # 用来存储每种类型节点的输出结果。
        for node_type, x_node in x_dict.items():
            out_dict[node_type] = []
        for edge_type, edge_index in edge_index_dict.items(): # 对于每种边类型，使用相应的GATConv层来计算节点特征。
            src_type, _, dst_type = edge_type
            edge_type = '__'.join(edge_type)
            out = self.gat_dict[edge_type](x_dict[src_type],
                                           edge_index,
                                           num_nodes = num_nodes_dict[dst_type])
            out = tlx.relu(out)
            out_dict[dst_type].append(out)
      # 在对所有边类型的输出计算完成后，对每个节点类型的输出进行聚合。
        for node_type, outs in out_dict.items():
            outs = tlx.stack(outs)
            out_dict[node_type] = self.sem_att_aggr(outs)
        return out_dict
    
class HAN(tlx.nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 metadata,
                 hidden_channels=128,
                 heads=8):
        super().__init__(name=None)
        self.han_conv = HANConv(in_channels,
                                hidden_channels,
                                metadata,
                                heads=heads)
        self.lin = tlx.nn.Linear(in_features=hidden_channels*heads, 
                                 out_features=out_channels)

    def forward(self, x_dict, edge_index_dict, num_nodes_dict):
        x = self.han_conv(x_dict, edge_index_dict, num_nodes_dict)
        out = {}
        for node_type, _ in num_nodes_dict.items():
            out[node_type] = self.lin(x[node_type]) # 通过一个全连接层将结果映射到输出空间
        return out
net = HAN(
        in_channels=graph.x_dict['movie'].shape[1],
        out_channels=3,
        metadata=graph.metadata(),
        drop_rate=args.drop_rate,
        hidden_channels=args.hidden_dim,
        heads=args.heads,
        name='han',
    )

4. 定义损失函数：

In [ ]:
from tensorlayerx.model import WithLoss, TrainOneStep 
class SemiSpvzLoss(WithLoss):
    def __init__(self, net, loss_fn):
        super(SemiSpvzLoss,   self).__init__(backbone=net, loss_fn=loss_fn)
    def forward(self, data, y):
        logits = self.backbone_network(data['x'],
        data['edge_index'],
        None,
        data['num_nodes']
        )
        # 根据输入的节点特征、边的连接信息等数据计算出模型的输出（logits）
        train_logits = tlx.gather(logits, data['train_idx'])  
        # 通过tlx.gather从标签中选择出训练集的真实标签 
        train_y = tlx.gather(data['y'], data['train_idx']) 
        loss = self._loss_fn(train_logits, train_y)
        return loss

train_weights = net.trainable_weights
loss_func = SemiSpvzLoss(net,   tlx.losses.softmax_cross_entropy_with_logits)


5. 设置优化器：

In [ ]:
optimizer = tlx.optimizers.Adam(lr=args.lr, weight_decay=args.l2_coef)

6. 模型评估函数：

In [ ]:
def calculate_acc(logits, y, metrics):
    metrics.update(logits, y)
    rst = metrics.result()
    metrics.reset()
    return rst
metrics = tlx.metrics.Accuracy()


7. 模型训练流程：

In [ ]:
train_one_step = TrainOneStep(loss_func, optimizer, train_weights)
best_val_acc = 0
for epoch in range(args.n_epoch):
    net.set_train()
    train_loss = train_one_step(data, graph.y) # 进行每轮训练
    net.set_eval()
    logits = net(data['x'], data['edge_index'], None, data['num_nodes']) # 执行模型的前向传播计算，生成预测的输出（logits）
    val_logits = tlx.gather(logits, data['val_idx'])
    val_y = tlx.gather(data['y'], data['val_idx'])
    val_acc = calculate_acc(val_logits, val_y, metrics)# 计算验证集上的准确率。val_logits是验证集的预测值，val_y是验证集的真实标签，metrics是评估标准
    print("Epoch [{:0>3d}] ".format(epoch+1)\
        + " train loss: {:.4f}".format(train_loss.item())\
        + " val acc: {:.4f}".format(val_acc))
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        # 保留验证集上表现最好的模型参数,作为测试集采用的模型参数
        net.save_weights(args.best_model_path+net.name+".npz", format='npz_dict')

net.load_weights(args.best_model_path+net.name+".npz", format='npz_dict')
net.set_eval()
logits = net(data['x'], data['edge_index'], None, data['num_nodes'])
test_logits = tlx.gather(logits, data['test_idx'])
test_y = tlx.gather(data['y'], data['test_idx'])
test_acc = calculate_acc(test_logits, test_y, metrics)
print("Test acc: {:.4f}".format(test_acc))
